In [ ]:
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera
from scipy.signal import find_peaks, butter, filtfilt
from simple_pid import PID
from pwm import PWM
import ipywidgets as ipw
import time
import matplotlib.pyplot as plt
import skimage as ski
import numpy as np
import time
import threading

def setEngineSpeed(percent, pwm):
    if percent < 0:
        percent = 0
    if percent > 1:
        percent = 1
    newSpeed = int(1000000 * percent + 1000000)
    pwm.duty_cycle = newSpeed

def turn(percent, pwm):
    if percent < -1:
        percent = -1
    if percent > 1:
        percent = 1
    if percent < 0:
        pwm.duty_cycle = int(1450000 + 950000 * percent)
    else:
        pwm.duty_cycle = int(440000 * percent + 1450000)

# Set the speed to be a discounted rate of the maxSpeed
def getNewSpeed(turnPercent, maxSpeed):
    speed = maxSpeed - (turnPercent * 1.8*maxSpeed)
    if(speed < 0.17):
        speed = 0.17
    return speed

# Create the sliders to effect the values of maxSpeed, minSpeed, Kp, Ki, Kd. Have two buttons for stopping the program and taking pictures
maxSpeed = ipw.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.25, description='Max Speed')
display(maxSpeed)
minSpeed = ipw.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.17, description='Min Speed')
display(minSpeed)
Kp = ipw.FloatSlider(min=0.0, max=10.0, step=0.01, value=.10, description='Kp')
display(Kp)
Ki = ipw.FloatSlider(min=0.0, max=10.0, step=0.01, value=0.00, description='Ki')
display(Ki)
Kd = ipw.FloatSlider(min=0.0, max=10.0, step=0.01, value=0.10, description='Kd')
display(Kd)
running = ipw.ToggleButton(value=True, description='STOP', disabled=False, button_style='', tooltip='Description', icon='check')
display(running)
button = ipw.ToggleButton(value=False, description='Snap photo', disabled=False,button_style='', tooltip='Description', icon='check')
display(button)


In [ ]:

# Setup and export pwm0 and pwm1
pwm0 = PWM(0)
pwm1 = PWM(1)
pwm0.export()
pwm1.export()

pwm0.period = 20000000
pwm1.period = 20000000

setEngineSpeed(0, pwm1)
turn(0, pwm0)

pwm0.enable = True
pwm1.enable = True

time.sleep(5)

# START OF COMPUTER VISION
res = (640,480)
b, a = butter(3, 0.007)
camera = PiCamera()
camera.sensor_mode = 6
camera.resolution = res
camera.framerate = 60
rawCapture = PiYUVArray(camera, size=res)
stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)
whiteCenterLine = 0 # Know where the center is


def func():
    turning = 'n' # Indicate which direction it is turning
    output = 0 # result of the PID
    oldNumsPeaks = 1
    start_time = time.time()
    pid = PID(Kp.value, Ki.value, Kd.value, setpoint=0) # PID used to follow the line
    pid.sample_time = 0.016
    pid.output_limits = (-1, 1)
    pid.proportional_on_measurement = True
    for f in stream:
        curr_Kp = Kp.value
        curr_Ki = Ki.value
        curr_Kd = Kd.value
        if Kp.value != curr_Kp or curr_Ki != Ki.value or curr_Kd != Kd.value:
            pid = PID(Kp.value, Ki.value, Kd.value, setpoint=0) # PID used to follow the line
            pid.sample_time = 0.016
            pid.output_limits = (-1, 1)
            pid.proportional_on_measurement = True
        L = f.array[440, :, 0]
        rawCapture.truncate(0)
        Lf = filtfilt(b, a, L)

        # Find peaks which are higher than 0.5
        p = find_peaks(Lf, height=128)
        num_peaks = len(p[0])
        if num_peaks == 1:
            whiteCenterLine = p[0][0]
        elif num_peaks >= 2:
            whiteCenterLine = (p[0][0] + p[0][len(p)-1]) / 2
        elif (num_peaks == 0):
            pass # REMEMBER LAST TURN
        
        if num_peaks == 0:
            # print("I AM NOT SEEING PEAKS")
            if turning == 'l':
                output = 1
            elif turning == 'r':
                output = -1
        else:
            percentOff = (whiteCenterLine - 320) / 240
            #print("percentOff")
            #print(percentOff)
            if percentOff >= 0.5:
                setEngineSpeed(0.16,pwm1)
            output = pid(percentOff) # Get the output of the PID as a percent
            #print("output")
            #print(output)
            if whiteCenterLine < 320:
                turning = 'l'
            else:
                turning = 'r'

        newSpeed = getNewSpeed(abs(output), maxSpeed.value)

        if newSpeed <= maxSpeed.value:
            setEngineSpeed(newSpeed, pwm1)
        else:
            setEngineSpeed(maxSpeed.value, pwm1)
        #print("Output: ", output)
        turn(output, pwm0) # Turn the car
        oldNumPeaks = num_peaks

        if not running.value:
            running.value = True
            break
            
    # Stop the motor
    setEngineSpeed(0, pwm1)
    # Reposition the servo to 0
    turn(0, pwm0)
    time.sleep(.1)
    stream.close()
    rawCapture.close()
    camera.close()
    pwm0.enable = False
    pwm1.enable = False
t = threading.Thread(target=func)
t.start()

In [ ]:
stream.close()
rawCapture.close()
camera.close()